<h1 align = "center"><code>Internal Code</code> Processing of RAW Data</h1>

In [1]:
import os   # miscellaneous os interfaces
import sys  # configuring python runtime environment

In [2]:
import datetime as dt

In [3]:
from copy import deepcopy

In [4]:
import numpy as np
import pandas as pd

%precision 3
pd.set_option('display.max_rows', 50) # max. rows to show
pd.set_option('display.max_columns', 15) # max. cols to show
np.set_printoptions(precision = 3, threshold = 15) # set np options
pd.options.display.float_format = '{:,.3f}'.format # float precisions

In [5]:
sys.path.append(os.path.join("..", "src", "engine")) # derivative engines for model control
from scaler import UnivariateRangedScaler # 🎉 scaling data with a defined data-range

In [10]:
# also append the `utilities` directory for additional helpful codes
sys.path.append(os.path.join("..", "utilities"))
from featuring import CreateSequence

In [6]:
ROOT = ".." # the document root is one level up, that contains all code structure
PROCESSED_DATA = os.path.join("data", "processed") # save the processed file in this directory

In [7]:
MARKET_SNAPSHOT_FILE_PATH = r"E:\database\Indian Energy Exchange\Day Ahead Market (DAM)\PROCESSED_MarketSnapshot_01-04-2012_31-12-2022.xlsx"
market_snapshot = pd.read_excel(MARKET_SNAPSHOT_FILE_PATH, sheet_name = "MarketSnapshot")
market_snapshot["EffectiveDate"] = pd.to_datetime(market_snapshot["EffectiveDate"], format = "%Y-%m-%d")

# already known that 01-08-2012 data records are missing from data source, and
# this is a very old record, thus just copy paste the previous days records
missing_records = deepcopy(market_snapshot[market_snapshot["EffectiveDate"] == dt.datetime(year = 2022, month = 7, day = 31)])
missing_records["EffectiveDate"] = pd.Timestamp(year = 2022, month = 7, day = 31)

market_snapshot = pd.concat([market_snapshot, missing_records], ignore_index = True)
market_snapshot.sort_values(by = ["EffectiveDate", "BlockID"], inplace = True)

# insert additional columns like year, month and day
market_snapshot["year"], market_snapshot["month"], market_snapshot["day"] = zip(*market_snapshot["EffectiveDate"].apply(lambda x : (x.year, x.month, x.day)))
market_snapshot = market_snapshot[["EffectiveDate", "year", "month", "day", "BlockID", "PurchaseBid", "SellBid", "MCV", "MCP"]]

market_snapshot.sample()

,EffectiveDate,year,month,day,BlockID,PurchaseBid,SellBid,MCV,MCP
47461,2013-08-09,2013,8,9,38,"3,516.900","6,398.800","3,106.630","2,250.580"


## Data Scaling

Currently, let's define a basic model considering only a univariate time series data of *market clearing price* for which we scale the price considering the minimum and maximum allowed price for that particular day. Interestingly, minimum price has always been ₹ 0.10 / MW while the maximum price is:

$$
p_b \in
\begin{cases}
    ₹ 20.00, \text{upto 03.04.2022} \\
    ₹ 12.00, \text{from 04.04.2022}
\end{cases}
$$

Considering the above use case in mind, the `MinMaxScaler` has been tweaked to accept `x_min` and `x_max` parameters, while working with a univariate time series data like this. Check documentation using **`help(UnivariateRangedScaler)`** for more information. The general scaling formula used as:

$$
    \hat{x} = \tau_0 + \frac{x - x_{min}}{x_{max} - x_{min}}
$$

In [8]:
scaler0 = UnivariateRangedScaler(x_min = 0.10 * 1e3, x_max = 20.00 * 1e3, feature_range = (1, 2))
mcp_values_0 = market_snapshot[market_snapshot["EffectiveDate"] <= dt.datetime(year = 2022, month = 4, day = 3)]["MCP"].values
sc_mcp_values_0 = scaler0.fit_transform(mcp_values_0)

scaler1 = UnivariateRangedScaler(x_min = 0.10 * 1e3, x_max = 12.00 * 1e3, feature_range = (1, 2))
mcp_values_1 = market_snapshot[market_snapshot["EffectiveDate"] >= dt.datetime(year = 2022, month = 4, day = 4)]["MCP"].values
sc_mcp_values_1 = scaler1.fit_transform(mcp_values_1)

market_snapshot["scaled(MCP)"] = np.concatenate((sc_mcp_values_0, sc_mcp_values_1))
market_snapshot.sample()

,EffectiveDate,year,month,day,BlockID,PurchaseBid,SellBid,MCV,MCP,scaled(MCP)
106721,2015-04-18,2015,4,18,66,"3,885.200","6,128.800","3,743.400","2,479.540",1.120


In [9]:
# market_snapshot.to_pickle(os.path.join(PROCESSED_DATA, "df_consolidated_2012-04-01_2022-12-31.pickle"))

## Data Formatting

Formatting the `data` into `x_train` and `y_train` using `.reshape()` functionality as required by LSTM network.

In [12]:
START_DATE = dt.datetime(year = 2019, month = 4, day = 1)
FINAL_DATE = dt.datetime(year = 2022, month = 10, day = 31)

In [13]:
data = market_snapshot[(market_snapshot["EffectiveDate"] >= START_DATE) & (market_snapshot["EffectiveDate"] <= FINAL_DATE)]["scaled(MCP)"].values
print(f"Current Data Shape: {data.shape}")

Current Data Shape: (125856,)


In [11]:
n_lookback = 3 * 96 # 🧪 look into 7 days past records
n_forecast = 2 * 96 # on t(-1) we want prediction for t(+1)

In [14]:
x_train, y_train = CreateSequence(data).create_univariate_series(n_lookback = n_lookback,n_forecast = n_forecast)
print(f"Formatted Data Shape: {x_train.shape, y_train.shape}")

Formatted Data Shape: ((125377, 288, 1), (125377, 192, 1))


In [ ]:
# training_data_outfile = os.path.join(
#     PROCESSED_DATA,
#     f"xy_train_consolidated_{str(START_DATE.date())}_{str(FINAL_DATE.date())}_n_lookback_{n_lookback}_n_forecast_{n_forecast}.npy"
# )

# with open(training_data_outfile, "wb") as fileObj:
#     np.save(fileObj, x_train)
#     np.save(fileObj, y_train)